In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMAGE_SIZE = (224, 224)  # Target image size for resizing
BATCH_SIZE = 32

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data Loading and Preprocessing
train_generator = train_datagen.flow_from_directory(
    'dataset/training',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # for multi-class classification
)

test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for test data
test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # for multi-class classification
)


Found 72 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [5]:
# Get the class labels
class_labels = train_generator.class_indices

# Print the class labels
print("Class labels:", class_labels)

# Invert the dictionary to get class names from indices
labels = dict((v, k) for k, v in class_labels.items())
print("Labels:", labels)


Class labels: {'ali': 0, 'nathan': 1}
Labels: {0: 'ali', 1: 'nathan'}


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Adjust the number of units according to your classification task
])


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE
)


2024-02-08 13:21:53.849558: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2024-02-08 13:21:53.877031: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2024-02-08 13:21:53.922336: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.


Epoch 1/10


2024-02-08 13:21:58.366036: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2024-02-08 13:21:58.379664: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.


2/2 [==============================] - 22s 12s/step - loss: 0.9484 - accuracy: 0.6406 - val_loss: 4.0864 - val_accuracy: 0.4688
Epoch 2/10
2/2 [==============================] - 13s 5s/step - loss: 3.6364 - accuracy: 0.3250 - val_loss: 0.5764 - val_accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 11s 4s/step - loss: 0.6480 - accuracy: 0.5750 - val_loss: 0.5935 - val_accuracy: 0.5312
Epoch 4/10
2/2 [==============================] - 8s 4s/step - loss: 0.6475 - accuracy: 0.4250 - val_loss: 0.5332 - val_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 9s 8s/step - loss: 0.5734 - accuracy: 0.7500 - val_loss: 0.4853 - val_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 8s 7s/step - loss: 0.5729 - accuracy: 0.6750 - val_loss: 0.5198 - val_accuracy: 0.4375
Epoch 7/10
2/2 [==============================] - 8s 4s/step - loss: 0.5145 - accuracy: 0.7500 - val_loss: 0.2143 - val_accuracy: 1.0000
Epoch 8/10
2/2 [==============================] 

In [7]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


1/1 [==============================] - 3s 3s/step - loss: 0.0182 - accuracy: 1.0000
Test Loss: 0.01816173642873764
Test Accuracy: 1.0


In [8]:
import numpy as np

# Generate predictions for the test set
predictions = model.predict(test_generator, steps=test_generator.samples // BATCH_SIZE)

# Convert the predicted probabilities to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get the class labels
class_labels = list(test_generator.class_indices.keys())

# Print some example predictions
for i in range(10):
    print("Predicted class:", class_labels[predicted_classes[i]])
    print("True class:", class_labels[true_classes[i]])
    print()  # Empty line for clarity


1/1 [==============================] - 3s 3s/step
Predicted class: ali
True class: ali

Predicted class: nathan
True class: ali

Predicted class: nathan
True class: ali

Predicted class: nathan
True class: ali

Predicted class: ali
True class: ali

Predicted class: nathan
True class: ali

Predicted class: nathan
True class: ali

Predicted class: nathan
True class: ali

Predicted class: nathan
True class: ali

Predicted class: ali
True class: ali



In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = '/home/nathan/Documents/TP_one_ML/ali_v.jpeg'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize the pixel values

# Make a prediction
prediction = model.predict(img_array)

# Convert the predicted probabilities to class labels
predicted_class = np.argmax(prediction)

# Get the class label
class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class]

# Display the prediction
print("Predicted class:", predicted_label)
print("Confidence score:", prediction[0][predicted_class])


1/1 [==============================] - 0s 39ms/step
Predicted class: ali
Confidence score: 0.98438466
